# COVID-19 knowledge graph

Relations between chemicals, genes and disease offer insights into the mechanisms behind higher order biochemical phenomena, such as drug-drug interactions, drug response and gene-disease associations. This work focused on extracting relations between chemicals, genes and disease from research articles to build a biomedical knowledge graph related to COVID-19.

In [ ]:
import numpy as np 
import pandas as pd 
import os

In [ ]:
!pip install pyecharts

In [ ]:
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from pyecharts import options as opts
from pyecharts.charts import Graph
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from keras.preprocessing import sequence
from transformers import BertTokenizer, BertForSequenceClassification

import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

**Load marked sentences**
These sentences containing more than two biomedical entities were extracted from abstracts and stored in a csv file. We combined the pairwise entities in each sentence to build relations.

In [ ]:
marked_sentence_df = pd.read_csv('/kaggle/input/covid19-knowledge-graphs-relation-files/marked_sentence/marked_sentence.csv')
marked_sentence_df.sample(15)

In [ ]:
def Build_graph(df,relation=False,repulsion=40,title='COVID-19 knowledge graph',labelShow=False):
    entity_type_dic = dict(df.drop_duplicates(['start_entity']).set_index(['start_entity'])['start_entity_type'])
    entity_type_dic.update(dict(df.drop_duplicates(['end_entity']).set_index(['end_entity'])['end_entity_type']))
    color = {'Disease':'#FF7F50','Gene':'#48D1CC','Chemical':'#B3EE3A'}
    cate =  {'Disease':0,'Gene':1,'Chemical':2}
    categories = [{'name':'Disease','itemStyle': {'normal': {'color': color['Disease']}}},{'name':'Gene','itemStyle': {'normal': {'color': color['Gene']}}},{'name':'Chemical','itemStyle': {'normal': {'color': color['Chemical']}}}]
    nodes = []
    for entity in list(set(df['start_entity'])|set(df['end_entity'])):
        nodes.append({'name': entity, 'symbolSize': max(10,np.log1p(df.loc[(df['start_entity']==entity)|(df['end_entity']==entity)].shape[0])*10//1),
                     'category':cate[entity_type_dic[entity]]})
    links = []
    for i in df.index:
        if not relation:
            links.append({'source': df.loc[i,'start_entity'], 'target': df.loc[i,'end_entity'],'value':df.loc[i,'marked_sentence']})
        else:
            links.append({'source': df.loc[i,'start_entity'], 'target': df.loc[i,'end_entity'], 'value':df.loc[i,'theme']})
    g = (
        Graph()
        .add('', nodes, links,categories, repulsion=repulsion,label_opts=opts.LabelOpts(is_show=labelShow))
        .set_global_opts(title_opts=opts.TitleOpts(title=title),legend_opts=opts.LegendOpts(orient='vertical', pos_left='2%', pos_top='40%',legend_icon='circle'))
        .render_notebook()
        )
    g1 = (
        Graph()
        .add('', nodes, links,categories, repulsion=repulsion,label_opts=opts.LabelOpts(is_show=labelShow))
        .set_global_opts(title_opts=opts.TitleOpts(title=title),legend_opts=opts.LegendOpts(orient='vertical', pos_left='2%', pos_top='40%',legend_icon='circle'))
        .render(path=title+'.html')
        )
    return g


In [ ]:
g = Build_graph(marked_sentence_df.sample(800),title='subsample of topology graph')
g

# Classify relations
Percha B 2018 revealed 10 broad themes for chemical-gene relations, 7 for chemical-disease, 10 for gene-disease and 9 for gene–gene in Medline abstracts. We used these labeled sentences to train a model to classify sentences in this work. As we cannot confirm order of entities, we predicted two possibilities(init_pred and reverse_pred) and retrieved the higher one. The details can be seen here

In [ ]:
class Args:
    task_type = 'chemical-disease'
    max_seq_len = 64
    bs = 64

class Conf:
    # some information can be found in:
    # Percha B, Altman R B. A global network of biomedical relationships derived from text[J]. Bioinformatics, 2018, 34(15): 2614-2624.
    relation_type = {'chemical-disease':['T', 'C', 'Sa', 'Pr', 'Pa', 'J'],
                     'disease-chemical':['Mp'],
                     'chemical-gene':['A+', 'A-', 'B', 'E+', 'E-', 'E', 'N'],
                     'gene-chemical':['O', 'K', 'Z'],
                     'gene-disease':['U', 'Ud', 'D', 'J', 'Te', 'Y', 'G'],
                     'disease-gene':['Md', 'X', 'L'],
                     'gene-gene':['B', 'W', 'V+', 'E+', 'E', 'I', 'H', 'Rg', 'Q'],
                     }

args = Args()
conf = Conf()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# load pretrained Bert model
def Bert_model(taskType,bertPath):
    label_df = pd.read_csv('/kaggle/input/covid19-knowledge-graphs-relation-files/%s_label.csv'%taskType)
    tokenizer = BertTokenizer.from_pretrained(bertPath,do_lower_case=False)
    model = BertForSequenceClassification.from_pretrained(bertPath, num_labels=label_df['label'].nunique())
    return label_df,tokenizer,model

# bulid data loader
def Data_loader(x,y=None,bs=128,shuffle=False,numWorkers=0):
    if y is not None:
        data = TensorDataset(x,y)
    else:
        data = TensorDataset(x)
    data_loader = DataLoader(dataset=data,batch_size=bs,shuffle=shuffle,num_workers=numWorkers)
    return data_loader

def Prepare_predict_data(tokenizer,bs):
    marked_sentences = marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[0])&\
                                              (marked_sentence_df['end_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[1]),'marked_sentence']
    # convert tokens to ids
    ids = marked_sentences.apply(lambda x:tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))).tolist()
    # padding ids
    ids = sequence.pad_sequences(ids,args.max_seq_len, truncating='post', padding='post')
    # we cannot confirm order of entities, so predict two possibilities
    reverse_marked_sentences = marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[0])&\
                                              (marked_sentence_df['end_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[1]),'marked_sentence']\
                                              .apply(lambda x:x.replace('start_entity','init_start_entity').replace('end_entity','start_entity').replace('init_start_entity','end_entity'))
    reverse_ids = reverse_marked_sentences.apply(lambda x:tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))).tolist()
    reverse_ids = sequence.pad_sequences(reverse_ids,args.max_seq_len, truncating='post', padding='post')
    predict_data_loader = Data_loader(torch.LongTensor(ids),torch.LongTensor(reverse_ids),bs=bs)
    return marked_sentences.values,predict_data_loader

def Predict():
    reverse_task_type = args.task_type.split('-')[1] + '-' + args.task_type.split('-')[0]
    def Filter(x):
        if x['init_pred'] in conf.relation_type[args.task_type]:
            if x['reverse_pred'] not in conf.relation_type[reverse_task_type]:
                # init_pred is a correct relation but reverse_pred not
                return 'init_pred'
            else:
                # init_pred and reverse_pred both are correct relations
                if x['init_pred_prob'] >= x['reverse_pred_prob']:
                    # init_pred_prob greater than or equal to reverse_pred_prob
                    return 'init_pred'
                else:
                    return 'reverse_pred'
        else:
            if x['reverse_pred'] not in conf.relation_type[reverse_task_type]:
                # init_pred and reverse_pred both are uncorrect relations
                return 'uncorrect'
            else:
                # reverse_pred is a correct relation but init_pred not
                return 'reverse_pred'
    label_df,tokenizer,model = Bert_model(args.task_type,'/kaggle/input/covid19-knowledge-graphs-relation-files/%s/'%args.task_type)
    marked_sentences,predict_data_loader = Prepare_predict_data(tokenizer,args.bs)
    model = model.to(device)
    preds = []
    preds_prob = []
    reverse_preds = []
    reverse_preds_prob = []
    for data in tqdm(predict_data_loader):
        ids,reverse_ids = [t.to(device) for t in data]
        outputs = model(input_ids=ids)
        logits = outputs[0]
        pred_prob, pred = torch.max(F.softmax(logits.data,1), 1)
        preds.extend(list(pred.cpu().detach().numpy()))
        preds_prob.extend(list(pred_prob.cpu().detach().numpy()))
        reverse_outputs = model(input_ids=reverse_ids)
        reverse_logits = reverse_outputs[0]
        reverse_pred_prob, reverse_pred = torch.max(F.softmax(reverse_logits.data,1), 1)
        reverse_preds.extend(list(reverse_pred.cpu().detach().numpy()))
        reverse_preds_prob.extend(list(reverse_pred_prob.cpu().detach().numpy()))

    pred_df = pd.DataFrame({'marked_sentence':marked_sentences,'init_pred':preds,'init_pred_prob':preds_prob,'reverse_pred':reverse_preds,'reverse_pred_prob':reverse_preds_prob})
    # map label(0, 1, 2...) to raw label(T, C, Sa...) 
    pred_df['init_pred'] = pred_df['init_pred'].replace(dict(label_df.set_index(['label'])['label_raw']))
    pred_df['reverse_pred'] = pred_df['reverse_pred'].replace(dict(label_df.set_index(['label'])['label_raw']))
    # judge the order of a pair of entities
    pred_df['filter'] = pred_df.apply(lambda x:Filter(x), axis=1)
    pred_df['pred'] = pred_df['init_pred']
    pred_df['pred_prob'] = pred_df['init_pred_prob']
    pred_df.loc[pred_df['filter']=='reverse_pred','pred'] = pred_df.loc[pred_df['filter']=='reverse_pred','reverse_pred']
    pred_df.loc[pred_df['filter']=='reverse_pred','pred_prob'] = pred_df.loc[pred_df['filter']=='reverse_pred','reverse_pred_prob']
    pred_df = pred_df.loc[pred_df['filter']!='uncorrect']
    pred_df = marked_sentence_df.merge(pred_df,how='inner',on='marked_sentence')
    pred_df['init_start_entity'] = pred_df['start_entity']
    pred_df['init_start_entity_type'] = pred_df['start_entity_type']
    pred_df.loc[pred_df['filter']=='reverse_pred','start_entity'] = pred_df.loc[pred_df['filter']=='reverse_pred','end_entity']
    pred_df.loc[pred_df['filter']=='reverse_pred','start_entity_type'] = pred_df.loc[pred_df['filter']=='reverse_pred','end_entity_type']
    pred_df.loc[pred_df['filter']=='reverse_pred','end_entity'] = pred_df.loc[pred_df['filter']=='reverse_pred','init_start_entity']
    pred_df.loc[pred_df['filter']=='reverse_pred','end_entity_type'] = pred_df.loc[pred_df['filter']=='reverse_pred','init_start_entity_type']
    pred_df.drop(['init_start_entity','init_start_entity_type'],axis=1,inplace=True)
    torch.cuda.empty_cache()
    return label_df,pred_df
# chemical-disease relation prediction
args.task_type = 'chemical-disease'
import pickle

try:
    c_d_label_df,c_d_pred_df = pickle.load(open("/kaggle/input/covid19-knowledge-graphs-relation-files/chem_disease.pickle", "rb"))
except (OSError, IOError) as e:
    c_d_label_df,c_d_pred_df = Predict()
    pickle.dump((c_d_label_df,c_d_pred_df), open("chem_disease.pickle", "wb"))

In [ ]:
# chemical-disease relation theme
c_d_label_df.rename(columns={'label_raw': 'pred'}, inplace=True)
c_d_label_df

In [ ]:
# chemical-disease classification results
c_d_pred_df=c_d_pred_df.merge(c_d_label_df, on='pred')
c_d_pred_df.sample(5)

In [ ]:
# chemical-gene relation prediction
args.task_type = 'chemical-gene'
try:
    c_g_label_df,c_g_pred_df = pickle.load(open("/kaggle/input/covid19-knowledge-graphs-relation-files/chem_gene.pickle", "rb"))
except (OSError, IOError) as e:
    c_g_label_df,c_g_pred_df = Predict()
    pickle.dump((c_g_label_df,c_g_pred_df), open("chem_gene.pickle", "wb"))

In [ ]:
# chemical-gene relation theme
c_g_label_df.rename(columns={'label_raw': 'pred'}, inplace=True)
c_g_label_df

In [ ]:
# chemical-gene classification results
c_g_pred_df=c_g_pred_df.merge(c_g_label_df, on='pred')
c_g_pred_df.sample(5)

In [ ]:
# gene-disease relation prediction
args.task_type = 'gene-disease'
try:
    g_d_label_df,g_d_pred_df = pickle.load(open("/kaggle/input/covid19-knowledge-graphs-relation-files/gene_disease.pickle", "rb"))
except (OSError, IOError) as e:
    g_d_label_df,g_d_pred_df = Predict()
    pickle.dump((g_d_label_df,g_d_pred_df), open("gene_disease.pickle", "wb"))

In [ ]:
# gene-disease relation theme
g_d_label_df.rename(columns={'label_raw': 'pred'}, inplace=True)
g_d_label_df

In [ ]:
# gene-disease classification results
g_d_pred_df=g_d_pred_df.merge(g_d_label_df, on='pred')
g_d_pred_df.sample(5)

In [ ]:
# gene-gene relation prediction
args.task_type = 'gene-gene'
try:
    g_g_label_df,g_g_pred_df = pickle.load(open("/kaggle/input/covid19-knowledge-graphs-relation-files/gene_gene.pickle", "rb"))
except (OSError, IOError) as e:
    g_g_label_df,g_g_pred_df = Predict()
    pickle.dump((g_g_label_df,g_g_pred_df), open("gene_gene.pickle", "wb"))

In [ ]:
# gene-gene relation theme
g_g_label_df.rename(columns={'label_raw': 'pred'}, inplace=True)
g_g_label_df

In [ ]:
# gene-gene classification results
g_g_pred_df=g_g_pred_df.merge(g_g_label_df, on='pred')
g_g_pred_df.sample(5)

# Show relation between covid-19 and other entities
pycharts cannot show multiple relations between two entities, so the graph is incomplete. Complete relations can be seen in pred_df.

In [ ]:
# chemical-COVID-19 relations
g = Build_graph(c_d_pred_df.loc[(c_d_pred_df['start_entity']=='COVID-19')|(c_d_pred_df['end_entity']=='COVID-19')],relation=True,repulsion=800,title='chemical-COVID-19 knowledge graph',labelShow=True)
g

In [ ]:
# gene-COVID-19 relations
g = Build_graph(g_d_pred_df.loc[(g_d_pred_df['start_entity']=='COVID-19')|(g_d_pred_df['end_entity']=='COVID-19')],relation=True,repulsion=60,title='gene-COVID-19 knowledge graph',labelShow=True)
g

In [ ]:
marked_sentence_df

In [ ]:
# gene-COVID-19 relations
g = Build_graph(marked_sentence_df.loc[(marked_sentence_df['start_entity']=='COVID-19')&(marked_sentence_df['end_entity_type']=='Disease')|(marked_sentence_df['start_entity_type']=='Disease')&(marked_sentence_df['end_entity']=='COVID-19')],relation=False,repulsion=60,title='disease-COVID-19 knowledge graph',labelShow=True)
g

# Answering researchers questions

In [ ]:
# merge all relation prediction and save results
cols = ['start_entity','end_entity','start_entity_type','end_entity_type','marked_sentence','pred','pred_prob','theme']
relation_df = pd.concat([c_d_pred_df[cols],c_g_pred_df[cols],g_d_pred_df[cols],g_g_pred_df[cols]]).append(marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].isin(['Chemical','Disease'])&(marked_sentence_df['start_entity_type']==marked_sentence_df['end_entity_type']))]).reset_index(drop=True)
relation_df.loc[(relation_df['pred'].isna())&(relation_df['start_entity_type']=='Chemical'),'pred'] = 'CC'
relation_df.loc[(relation_df['pred'].isna())&(relation_df['start_entity_type']=='Disease'),'pred'] = 'DD'
relation_df = relation_df[cols]
relation_df.to_csv('relation.csv',index=False)

In [ ]:
labels_df= pd.concat([c_d_label_df,c_g_label_df,g_d_label_df,g_g_label_df])

In [ ]:
# subsample of Combined knowledge graph
temp=relation_df.loc[(relation_df['start_entity']=='COVID-19')|(relation_df['end_entity']=='COVID-19')]
temp = temp[cols]
g = Build_graph(temp.sample(200),relation=True,repulsion=15,title='subsample of COVID-19 knowledge graph',labelShow=True)
g

# Knowledge graph between pair of entities(Covid specific and Non specific also)

In [ ]:
ch1='Chemical'
ch2='Disease'
ch3=2

In [ ]:
ch1=input('enter choice 1 : Chemical,Gene,Disease = You selected ')
ch2=input('enter choice 2 : Chemical,Gene,Disease = You selected ')
ch3=int(input('enter 2-Covid19 related OR  1- Not specific to Covid  = You selected '))
if ch3==1:
    temp=relation_df.loc[((relation_df['start_entity_type']==ch1)&(relation_df['end_entity_type']==ch2))|((relation_df['start_entity_type']==ch2)&(relation_df['end_entity_type']==ch1))]#
    temp = temp[cols]
    g = Build_graph(temp,relation=True,repulsion=15,title= ch1+'_'+ch2+'_knowledge graph',labelShow=True)
    temp.to_csv(ch1+'_'+ch2+'_knowledge graph.csv')
else:
    temp=relation_df.loc[((relation_df['start_entity_type']==ch1)&(relation_df['end_entity_type']==ch2))|((relation_df['start_entity_type']==ch2)&(relation_df['end_entity_type']==ch1))]#
    temp=temp.loc[(relation_df['start_entity']=='COVID-19')|(relation_df['end_entity']=='COVID-19')]
    temp = temp[cols]
    g = Build_graph(temp,relation=True,repulsion=15,title= 'Covid19 Specific_'+ch1+'_'+ch2+'_knowledge graph',labelShow=True)
    temp.to_csv('Covid19 Specific_'+ch1+'_'+ch2+'_knowledge graph.csv')
g

# Search based on desired relationships

In [ ]:
labels_df

In [ ]:
import re
ch5='Mutations affect disease course'
# ch5=input('enter desired relationship')
temp=relation_df.loc[(relation_df['theme']==ch5)]#
if temp is None:
    print('yes')
temp = temp[cols]
ch5=re.sub(r'/', '_', ch5)
g = Build_graph(temp,relation=True,repulsion=15,title=ch5+'_knowledge graph',labelShow=True)
temp.to_csv(ch5+'_knowledge graph.csv')
g